In [60]:
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import MySQLdb
import mysql.connector

#stationinfo_url = "https://gbfs.divvybikes.com/gbfs/en/station_information.json"
stationstatus_url = "https://gbfs.divvybikes.com/gbfs/en/station_status.json"

# Get country information in JSON format
#divvy_stationinfo_response = requests.get(stationinfo_url).json()
divvy_stationstatus_response = requests.get(stationstatus_url).json()

# First element is general information, second is countries themselves
#divvy_stationinfo_response = divvy_stationinfo_response["data"]["stations"]
divvy_stationstatus_response = divvy_stationstatus_response["data"]["stations"]

In [61]:
divvy_stationstatus_response

[{'station_id': '2',
  'num_bikes_available': 0,
  'num_ebikes_available': 0,
  'num_bikes_disabled': 0,
  'num_docks_available': 39,
  'num_docks_disabled': 0,
  'is_installed': 1,
  'is_renting': 1,
  'is_returning': 1,
  'last_reported': 1576797052,
  'eightd_has_available_keys': False},
 {'station_id': '3',
  'num_bikes_available': 3,
  'num_ebikes_available': 0,
  'num_bikes_disabled': 0,
  'num_docks_available': 52,
  'num_docks_disabled': 0,
  'is_installed': 1,
  'is_renting': 1,
  'is_returning': 1,
  'last_reported': 1576803409,
  'eightd_has_available_keys': False},
 {'station_id': '4',
  'num_bikes_available': 6,
  'num_ebikes_available': 0,
  'num_bikes_disabled': 0,
  'num_docks_available': 17,
  'num_docks_disabled': 0,
  'is_installed': 1,
  'is_renting': 1,
  'is_returning': 1,
  'last_reported': 1576793647,
  'eightd_has_available_keys': False},
 {'station_id': '5',
  'num_bikes_available': 12,
  'num_ebikes_available': 0,
  'num_bikes_disabled': 0,
  'num_docks_avail

In [62]:
#divvy_stationinfo_df = pd.DataFrame.from_dict(divvy_stationinfo_response, orient='columns')
#divvy_stationinfo_df.reset_index(level=0, inplace=True)
#clean_divvy_stationinfo_df = divvy_stationinfo_df.drop(["rental_methods"], axis = 1)
#divvy_stationinfo_df.columns
#clean_divvy_stationinfo_df

In [63]:
divvy_stationinfo_df = pd.read_csv("Resources/divvy_neighborhood.csv")
divvy_stationinfo_df.head()

,Unnamed: 0,capacity,divvy_lat,divvy_lng,name,station_id,neighborhood,count
0,0,39,41.876511,-87.620548,Buckingham Fountain,2,Chicago Loop,29
1,1,55,41.867226,-87.615355,Shedd Aquarium,3,Near South Side,13
2,2,23,41.856268,-87.613348,Burnham Harbor,4,Near South Side,13
3,3,23,41.874053,-87.627716,State St & Harrison St,5,Chicago Loop,29
4,4,39,41.886976,-87.612813,Dusable Harbor,6,North Lake Shore Drive,1


In [64]:
divvy_stationstatus_df = pd.DataFrame.from_dict(divvy_stationstatus_response, orient='columns')
divvy_stationstatus_df.reset_index(level=0, inplace=True)
divvy_stationstatus_df = divvy_stationstatus_df.drop(["eightd_has_available_keys", "is_installed", "is_renting", "is_returning"], axis=1)
divvy_stationstatus_df['station_id'] = divvy_stationstatus_df['station_id'].astype(int)
divvy_stationstatus_df.head()

,index,last_reported,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,num_ebikes_available,station_id
0,0,1576797052,0,0,39,0,0,2
1,1,1576803409,3,0,52,0,0,3
2,2,1576793647,6,0,17,0,0,4
3,3,1576802236,12,0,11,0,0,5
4,4,1576800535,3,0,36,0,0,6


In [65]:
divvy_stationinfo_df.dtypes

Unnamed: 0        int64
capacity          int64
divvy_lat       float64
divvy_lng       float64
name             object
station_id        int64
neighborhood     object
count             int64
dtype: object

In [66]:
divvy_stationstatus_df.dtypes

index                   int64
last_reported           int64
num_bikes_available     int64
num_bikes_disabled      int64
num_docks_available     int64
num_docks_disabled      int64
num_ebikes_available    int64
station_id              int32
dtype: object

In [67]:
redfin_df = pd.read_csv("Resources/Combined_neighborhood_crime_data.csv")
redfin_df = redfin_df.drop(["Crime Count"], axis=1)
redfin_df = redfin_df.rename(columns={'City':'city', 'Neighborhood':'neighborhood', 'Lat':'redfin_lat','Lng':'redfin_lng', 'Month':'month', 'Sale Price': 'sale_price', 'Homes Sold':'homes_sold', 'New Listings':'new_listings', 'Days on Market':'days_on_market'})
redfin_df.head()

,Unnamed: 0,city,neighborhood,redfin_lat,redfin_lng,month,sale_price,homes_sold,new_listings,days_on_market
0,0,"Chicago, IL",Albany Park,41.968327,-87.728028,Aug-19,295.0,113,133,54
1,1,"Chicago, IL",Andersonville,41.981658,-87.668476,Aug-19,419.0,94,79,50
2,2,"Chicago, IL",Archer Heights,41.807900,-87.723585,Aug-19,226.0,20,31,26
3,3,"Chicago, IL",Armour Square,41.840755,-87.634019,Aug-19,265.0,7,20,58
4,4,"Chicago, IL",Ashburn,41.749690,-87.712007,Aug-19,199.0,100,137,53


In [68]:
database_path = "Project2.db" #whatever you type in here will be the name of the database
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [69]:
divvy_merged_df = divvy_stationstatus_df.merge(divvy_stationinfo_df, how='inner', on='station_id')
divvy_merged_df = divvy_merged_df.drop(["Unnamed: 0"], axis=1)
divvy_merged_df.head(50)

,index,last_reported,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,num_ebikes_available,station_id,capacity,divvy_lat,divvy_lng,name,neighborhood,count
0,0,1576797052,0,0,39,0,0,2,39,41.876511,-87.620548,Buckingham Fountain,Chicago Loop,29
1,1,1576803409,3,0,52,0,0,3,55,41.867226,-87.615355,Shedd Aquarium,Near South Side,13
2,2,1576793647,6,0,17,0,0,4,23,41.856268,-87.613348,Burnham Harbor,Near South Side,13
3,3,1576802236,12,0,11,0,0,5,23,41.874053,-87.627716,State St & Harrison St,Chicago Loop,29
4,4,1576800535,3,0,36,0,0,6,39,41.886976,-87.612813,Dusable Harbor,North Lake Shore Drive,1
5,5,1576804395,9,0,10,0,0,7,19,41.886349,-87.617517,Field Blvd & South Water St,Chicago Loop,29
6,6,1576799446,8,0,7,0,0,9,15,41.828792,-87.680604,Leavitt St & Archer Ave,McKinley Park,4
7,7,1576752988,1,0,10,0,0,11,11,41.766638,-87.576450,Jeffery Blvd & 71st St,Jackson Park Highlands,1
8,8,1576804035,5,0,10,0,0,12,15,41.766409,-87.565688,South Shore Dr & 71st St,South Shore,6
9,9,1576804028,3,0,32,0,0,13,35,41.932418,-87.652705,Wilton Ave & Diversey Pkwy,Sheffield & DePaul,5


In [70]:
redfin_divvy_merged_df = redfin_df.merge(divvy_merged_df, how='inner', on='neighborhood')
redfin_divvy_merged_df

,Unnamed: 0,city,neighborhood,redfin_lat,redfin_lng,month,sale_price,homes_sold,new_listings,days_on_market,...,num_bikes_disabled,num_docks_available,num_docks_disabled,num_ebikes_available,station_id,capacity,divvy_lat,divvy_lng,name,count
0,0,"Chicago, IL",Albany Park,41.968327,-87.728028,Aug-19,295.0,113,133,54,...,0,6,0,0,477,11,41.965875,-87.700825,Manor Ave & Leland Ave,2
1,0,"Chicago, IL",Albany Park,41.968327,-87.728028,Aug-19,295.0,113,133,54,...,0,7,0,0,495,11,41.961108,-87.728970,Keystone Ave & Montrose Ave,2
2,1,"Chicago, IL",Andersonville,41.981658,-87.668476,Aug-19,419.0,94,79,50,...,0,9,0,0,460,15,41.983593,-87.669154,Clark St & Bryn Mawr Ave,2
3,1,"Chicago, IL",Andersonville,41.981658,-87.668476,Aug-19,419.0,94,79,50,...,0,5,0,0,463,19,41.977997,-87.668047,Clark St & Berwyn Ave,2
4,3,"Chicago, IL",Armour Square,41.840755,-87.634019,Aug-19,265.0,7,20,58,...,2,2,0,0,132,15,41.850084,-87.632141,Wentworth Ave & 24th St (Temp),6
5,3,"Chicago, IL",Armour Square,41.840755,-87.634019,Aug-19,265.0,7,20,58,...,0,10,0,0,218,11,41.856802,-87.633879,Wells St & 19th St,6
6,3,"Chicago, IL",Armour Square,41.840755,-87.634019,Aug-19,265.0,7,20,58,...,0,8,0,0,401,11,41.842733,-87.635491,Shields Ave & 28th Pl,6
7,3,"Chicago, IL",Armour Square,41.840755,-87.634019,Aug-19,265.0,7,20,58,...,0,12,0,0,402,15,41.838464,-87.635406,Shields Ave & 31st St,6
8,3,"Chicago, IL",Armour Square,41.840755,-87.634019,Aug-19,265.0,7,20,58,...,0,10,0,0,403,15,41.834530,-87.631823,Wentworth Ave & 33rd St,6
9,3,"Chicago, IL",Armour Square,41.840755,-87.634019,Aug-19,265.0,7,20,58,...,0,9,0,0,405,15,41.830777,-87.632504,Wentworth Ave & 35th St,6


In [74]:
redfin_divvy_merged_df.to_csv("redfin_divvy_merged_data.csv")

In [71]:
inspector = inspect(engine)
inspector.get_table_names() #nice we just created our own db and its empty as it should be!

['divvy_stationinfo', 'divvy_stationstatus', 'redfin_aug2019']

In [72]:
divvy_stationstatus_df.to_sql(name='divvy_stationstatus', con=engine, if_exists='replace', index=False)
clean_divvy_stationinfo_df.to_sql(name='divvy_stationinfo', con=engine, if_exists='replace', index=False)
redfin_df.to_sql(name='redfin_aug2019', con=engine, if_exists='replace', index=False)
redfin_divvy_merged_df.to_sql(name='merged_redfin_divvy', con=engine, if_exists='replace', index=False)

In [73]:
inspector = inspect(engine)
inspector.get_table_names()

['divvy_stationinfo',
 'divvy_stationstatus',
 'merged_redfin_divvy',
 'redfin_aug2019']